<a href="https://colab.research.google.com/github/algovritti/colab_research/blob/main/Long_Short_Position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Plane Short

In [ ]:
import pandas as pd
import numpy as np


data = pd.read_excel("future_data.xlsx")
#print(data.head())
# Remove spaces from column names
data.columns = data.columns.str.strip()

# Convert Date column to datetime
data["Date"] = pd.to_datetime(data["Date"])
start_date = "2025-01-01"
end_date = "2025-02-28"

mask = (data["Date"] >= start_date) & (data["Date"] <= end_date)
filtered_data = data.loc[mask]

#print(filtered_data.head())

trades = []
filtered_data.set_index("Date", inplace=True)

# --- Strategy Parameters ---


def backtest(sl):
    trades = []
    for date, row in filtered_data.iterrows():
        entry = row["Open"]
        stop  = entry * (1 + sl)             # SHORT stop
        exit_ = stop if row["High"] >= stop else row["Close"]
        pnl   = entry - exit_
        trades.append(pnl)

    # --- performance metrics ---
    total_pnl = np.sum(trades)
    total_trades = len(trades)

    cumulative_pnl = np.cumsum(trades)
    running_max = np.maximum.accumulate(cumulative_pnl)
    drawdown = cumulative_pnl - running_max
    max_dd = drawdown.min()

    return total_pnl, max_dd, total_trades


# 🔍 Compare across stop-loss values
for sl in [0.002, 0.005, 0.01, 0.02, 0.05]:
    total, max_dd, n_trades = backtest(sl)
    print(f"SHORT SL={sl*100:.2f}% → Total PnL={total:.2f}, Max DD={max_dd:.2f}, Trades={n_trades}")




SHORT SL=0.20% → Total PnL=774.14, Max DD=-464.43, Trades=41
SHORT SL=0.50% → Total PnL=1428.77, Max DD=-634.08, Trades=41
SHORT SL=1.00% → Total PnL=1516.04, Max DD=-488.85, Trades=41
SHORT SL=2.00% → Total PnL=1220.75, Max DD=-488.75, Trades=41
SHORT SL=5.00% → Total PnL=1220.75, Max DD=-488.75, Trades=41


## Long

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_excel("future_data.xlsx")
#print(data.head())
# Remove spaces from column names
data.columns = data.columns.str.strip()

# Convert Date column to datetime
data["Date"] = pd.to_datetime(data["Date"])
start_date = "2025-08-01"
end_date = "2025-08-30"

mask = (data["Date"] >= start_date) & (data["Date"] <= end_date)
filtered_data = data.loc[mask]

#print(filtered_data.head())

trades = []
filtered_data.set_index("Date", inplace=True)

def backtest_long(sl):
    trades = []
    for date, row in filtered_data.iterrows():
        entry = row["Open"]
        stop  = entry * (1 - sl)             # LONG stop (below entry)
        exit_ = stop if row["Low"] <= stop else row["Close"]
        pnl   = exit_ - entry
        trades.append(pnl)

    # --- performance metrics ---
    total_pnl = np.sum(trades)
    total_trades = len(trades)

    cumulative_pnl = np.cumsum(trades)
    running_max = np.maximum.accumulate(cumulative_pnl)
    drawdown = cumulative_pnl - running_max
    max_dd = drawdown.min()

    return total_pnl, max_dd, total_trades


# 🔍 Compare across stop-loss values
for sl in [0.002, 0.005, 0.01, 0.02, 0.05]:
    total, max_dd, n_trades = backtest_long(sl)
    print(f"LONG SL={sl*100:.2f}% → Total PnL={total:.2f}, Max DD={max_dd:.2f}, Trades={n_trades}")



LONG SL=0.20% → Total PnL=272.99, Max DD=-197.38, Trades=17
LONG SL=0.50% → Total PnL=-172.60, Max DD=-442.82, Trades=17
LONG SL=1.00% → Total PnL=-149.09, Max DD=-327.70, Trades=17
LONG SL=2.00% → Total PnL=-130.30, Max DD=-327.70, Trades=17
LONG SL=5.00% → Total PnL=-130.30, Max DD=-327.70, Trades=17


## Take Long or short based on open +- 5%

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np


data = pd.read_excel("future_data.xlsx")
#print(data.head())
# Remove spaces from column names
data.columns = data.columns.str.strip()

# Convert Date column to datetime
data["Date"] = pd.to_datetime(data["Date"])
start_date = "2025-01-01"
end_date = "2025-08-26"

mask = (data["Date"] >= start_date) & (data["Date"] <= end_date)
filtered_data = data.loc[mask]

def open_move_backtest(df, move_pct=0.005, stop_loss_pct=0.005, direction="both"):
    """
    Backtest rule:
      - Use the day's Open as reference.
      - If price moves up by >= move_pct from Open during the day (High >= Open*(1+move_pct)),
        enter LONG at entry_up = Open*(1+move_pct).
      - If price moves down by >= move_pct from Open during the day (Low <= Open*(1-move_pct)),
        enter SHORT at entry_down = Open*(1-move_pct).
      - If both triggers occur in the same day, take the one closer to Open (tie-break).
      - Exit intraday at stop (LONG: Low <= stop, SHORT: High >= stop), else exit at Close.
    Returns:
      trades_df, metrics_dict
    """
    tmp = df.copy()
    # ensure Date index
    if "Date" in tmp.columns:
        tmp = tmp.set_index("Date")
    tmp.index = pd.to_datetime(tmp.index)

    trades = []
    for date, row in tmp.iterrows():
        open_p = float(row["Open"])
        high_p = float(row["High"])
        low_p  = float(row["Low"])
        close_p = float(row["Close"])

        entry_up = open_p * (1 + move_pct)   # long trigger price
        entry_dn = open_p * (1 - move_pct)   # short trigger price

        long_hit  = high_p >= entry_up
        short_hit = low_p <= entry_dn

        # If both hit, choose the one closer to Open (tie-break)
        if long_hit and short_hit:
            dist_long = entry_up - open_p
            dist_short = open_p - entry_dn
            # choose the smaller distance (more likely occurred first)
            if dist_long <= dist_short:
                short_hit = False
            else:
                long_hit = False

        # LONG case
        if long_hit and (direction in ("both","long")):
            entry = entry_up
            stop = entry * (1 - stop_loss_pct)
            stop_hit = low_p <= stop
            exit_price = stop if stop_hit else close_p
            pnl = exit_price - entry
            trades.append([date, "LONG", entry, exit_price, pnl, stop_hit])

        # SHORT case
        elif short_hit and (direction in ("both","short")):
            entry = entry_dn
            stop = entry * (1 + stop_loss_pct)
            stop_hit = high_p >= stop
            exit_price = stop if stop_hit else close_p
            pnl = entry - exit_price
            trades.append([date, "SHORT", entry, exit_price, pnl, stop_hit])

        # else: no trade that day

    trades_df = pd.DataFrame(trades, columns=[
        "Date", "Type", "Entry", "Exit", "PnL_Points", "StopHit"
    ])
    if not trades_df.empty:
        trades_df.set_index("Date", inplace=True)

    # Metrics
    pnl_series = trades_df["PnL_Points"] if not trades_df.empty else pd.Series(dtype=float)
    total_pnl = float(pnl_series.sum()) if not pnl_series.empty else 0.0
    cumulative_pnl = pnl_series.cumsum()
    running_max = cumulative_pnl.cummax() if not cumulative_pnl.empty else cumulative_pnl
    drawdown = cumulative_pnl - running_max
    max_dd = float(drawdown.min()) if not drawdown.empty else 0.0
    total_trades = int(len(trades_df))
    wins = int((pnl_series > 0).sum()) if not pnl_series.empty else 0
    win_rate = float(wins / total_trades * 100) if total_trades > 0 else 0.0
    avg_pnl = float(pnl_series.mean()) if total_trades > 0 else 0.0

    metrics = {
        "Total_PnL": total_pnl,
        "Max_Drawdown": max_dd,
        "Total_Trades": total_trades,
        "Win_Rate_pct": win_rate,
        "Avg_PnL": avg_pnl
    }

    return trades_df, metrics

# -------------------------
# Example usage:
# -------------------------
# Ensure filtered_data exists and has Date/Open/High/Low/Close columns
# filtered_data["Date"] = pd.to_datetime(filtered_data["Date"])   # if needed
# filtered_data = filtered_data.set_index("Date")                 # optional

trades_df, metrics = open_move_backtest(filtered_data, move_pct=0.005, stop_loss_pct=0.005, direction="both")

print("Metrics:")
for k,v in metrics.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2f}")
    else:
        print(f"{k}: {v}")
# show trades
display(trades_df)


Metrics:
Total_PnL: -12513.24
Max_Drawdown: -12793.59
Total_Trades: 114
Win_Rate_pct: 3.51
Avg_PnL: -109.77


,Type,Entry,Exit,PnL_Points,StopHit
Date,,,,,
2025-01-01,LONG,23617.5000,23897.850000,280.350000,False
2025-01-02,LONG,24030.5550,23910.402225,-120.152775,True
2025-01-03,SHORT,24118.8000,24239.394000,-120.594000,True
2025-01-06,SHORT,24024.3745,24144.496372,-120.121872,True
2025-01-08,SHORT,23681.0000,23799.405000,-118.405000,True
...,...,...,...,...,...
2025-08-12,LONG,24728.1255,24604.484872,-123.640628,True
2025-08-18,LONG,25030.8315,24966.800000,-64.031500,False
2025-08-20,LONG,25099.7745,24974.275627,-125.498873,True


In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import yfinance as yf

data = yf.download("^NSEI", start="2025-07-01", end="2025-08-26", interval="1d")

#print(data
#data = pd.read_excel("future_data.xlsx")
#print(data.head())
# Remove spaces from column names
#data.columns = data.columns.str.strip()

# Convert Date column to datetime


#mask = (data["Date"] >= start_date) & (data["Date"] <= end_date)
filtered_data = data

def open_move_backtest(df, move_pct=0.005, stop_loss_pct=0.005, direction="both"):
    """
    Backtest rule:
      - Use the day's Open as reference.
      - If price moves up by >= move_pct from Open during the day (High >= Open*(1+move_pct)),
        enter LONG at entry_up = Open*(1+move_pct).
      - If price moves down by >= move_pct from Open during the day (Low <= Open*(1-move_pct)),
        enter SHORT at entry_down = Open*(1-move_pct).
      - If both triggers occur in the same day, take the one closer to Open (tie-break).
      - Exit intraday at stop (LONG: Low <= stop, SHORT: High >= stop), else exit at Close.
    Returns:
      trades_df, metrics_dict
    """
    tmp = df.copy()
    # ensure Date index
    if "Date" in tmp.columns:
        tmp = tmp.set_index("Date")
    tmp.index = pd.to_datetime(tmp.index)

    trades = []
    for date, row in tmp.iterrows():
        open_p = float(row["Open"])
        high_p = float(row["High"])
        low_p  = float(row["Low"])
        close_p = float(row["Close"])

        entry_up = open_p * (1 + move_pct)   # long trigger price
        entry_dn = open_p * (1 - move_pct)   # short trigger price

        long_hit  = high_p >= entry_up
        short_hit = low_p <= entry_dn

        # If both hit, choose the one closer to Open (tie-break)
        if long_hit and short_hit:
            dist_long = entry_up - open_p
            dist_short = open_p - entry_dn
            # choose the smaller distance (more likely occurred first)
            if dist_long <= dist_short:
                short_hit = False
            else:
                long_hit = False

        # LONG case
        if long_hit and (direction in ("both","long")):
            entry = entry_up
            stop = entry * (1 - stop_loss_pct)
            stop_hit = low_p <= stop
            exit_price = stop if stop_hit else close_p
            pnl = exit_price - entry
            trades.append([date, "LONG", entry, exit_price, pnl, stop_hit])

        # SHORT case
        elif short_hit and (direction in ("both","short")):
            entry = entry_dn
            stop = entry * (1 + stop_loss_pct)
            stop_hit = high_p >= stop
            exit_price = stop if stop_hit else close_p
            pnl = entry - exit_price
            trades.append([date, "SHORT", entry, exit_price, pnl, stop_hit])

        # else: no trade that day

    trades_df = pd.DataFrame(trades, columns=[
        "Date", "Type", "Entry", "Exit", "PnL_Points", "StopHit"
    ])
    if not trades_df.empty:
        trades_df.set_index("Date", inplace=True)

    # Metrics
    pnl_series = trades_df["PnL_Points"] if not trades_df.empty else pd.Series(dtype=float)
    total_pnl = float(pnl_series.sum()) if not pnl_series.empty else 0.0
    cumulative_pnl = pnl_series.cumsum()
    running_max = cumulative_pnl.cummax() if not cumulative_pnl.empty else cumulative_pnl
    drawdown = cumulative_pnl - running_max
    max_dd = float(drawdown.min()) if not drawdown.empty else 0.0
    total_trades = int(len(trades_df))
    wins = int((pnl_series > 0).sum()) if not pnl_series.empty else 0
    win_rate = float(wins / total_trades * 100) if total_trades > 0 else 0.0
    avg_pnl = float(pnl_series.mean()) if total_trades > 0 else 0.0

    metrics = {
        "Total_PnL": total_pnl,
        "Max_Drawdown": max_dd,
        "Total_Trades": total_trades,
        "Win_Rate_pct": win_rate,
        "Avg_PnL": avg_pnl
    }

    return trades_df, metrics

# -------------------------
# Example usage:
# -------------------------
# Ensure filtered_data exists and has Date/Open/High/Low/Close columns
# filtered_data["Date"] = pd.to_datetime(filtered_data["Date"])   # if needed
# filtered_data = filtered_data.set_index("Date")                 # optional

trades_df, metrics = open_move_backtest(filtered_data, move_pct=0.005, stop_loss_pct=0.005, direction="both")

print("Metrics:")
for k,v in metrics.items():
    if isinstance(v, float):
        print(f"{k}: {v:.2f}")
    else:
        print(f"{k}: {v}")
# show trades
display(trades_df)


/tmp/ipython-input-2104500032.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("^NSEI", start="2025-07-01", end="2025-08-26", interval="1d")
[*********************100%***********************]  1 of 1 completed

Metrics:
Total_PnL: -2612.77
Max_Drawdown: -2485.46
Total_Trades: 21
Win_Rate_pct: 0.00
Avg_PnL: -124.42



/tmp/ipython-input-2104500032.py:42: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  open_p = float(row["Open"])
/tmp/ipython-input-2104500032.py:43: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  high_p = float(row["High"])
/tmp/ipython-input-2104500032.py:44: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  low_p  = float(row["Low"])
/tmp/ipython-input-2104500032.py:45: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  close_p = float(row["Close"])


,Type,Entry,Exit,PnL_Points,StopHit
Date,,,,,
2025-07-02,SHORT,25460.359277,25587.661074,-127.301796,True
2025-07-10,SHORT,25384.092139,25511.012599,-126.920461,True
2025-07-11,SHORT,25129.222500,25254.868612,-125.646112,True
2025-07-14,SHORT,25023.752500,25148.871262,-125.118762,True
2025-07-15,LONG,25214.947500,25088.872762,-126.074737,True
2025-07-17,SHORT,25104.596250,25230.119231,-125.522981,True
2025-07-18,SHORT,24983.008027,25107.923067,-124.915040,True
2025-07-22,SHORT,25040.817139,25166.021224,-125.204086,True
2025-07-24,SHORT,25117.084277,25242.669699,-125.585421,True


# (5m intraday backtest)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

def backtest_open_move_intraday(symbol, start="2025-07-01", end="2025-08-26", move_pct=0.005, sl_pct=0.005):
    # --- Download daily data (for open reference) ---
    daily = yf.download(symbol, start=start, end=end, interval="1d")
    daily.index = pd.to_datetime(daily.index)

    # --- Download 5-min intraday data ---
    intraday = yf.download(symbol, start=start, end=end, interval="5m")
    intraday.index = pd.to_datetime(intraday.index)

    # Convert intraday index to IST
    intraday.index = intraday.index.tz_convert("Asia/Kolkata")

    # Add a date column for grouping
    intraday["Date"] = intraday.index.date
    daily["Date"] = daily.index.date

    trades = []

    # --- Loop through each trading day ---
    for d, row in daily.iterrows():
        day = row["Date"]
        day_open = row["Open"]

        # Get intraday candles for this date
        day_data = intraday[intraday["Date"] == day]
        if day_data.empty:
            continue  # skip holidays / missing data

        # Define thresholds
        long_trigger = day_open * (1 + move_pct)
        short_trigger = day_open * (1 - move_pct)
        trade_taken = False
        entry = None
        trade_type = None
        long_sl = short_sl = None

        for ts, candle in day_data.iterrows():
            high = candle["High"]
            low = candle["Low"]
            close = candle["Close"]

            # If not in trade, check for entry
            if not trade_taken:
                if high >= long_trigger:  # Long entry
                    entry = long_trigger
                    long_sl = entry * (1 - sl_pct)
                    trade_type = "LONG"
                    trade_taken = True
                elif low <= short_trigger:  # Short entry
                    entry = short_trigger
                    short_sl = entry * (1 + sl_pct)
                    trade_type = "SHORT"
                    trade_taken = True
                continue

            # If in trade, check stop loss
            if trade_taken:
                if trade_type == "LONG":
                    if low <= long_sl:  # SL hit
                        exit_price = long_sl
                        trades.append([day, trade_type, entry, exit_price, exit_price-entry])
                        break
                elif trade_type == "SHORT":
                    if high >= short_sl:  # SL hit
                        exit_price = short_sl
                        trades.append([day, trade_type, entry, exit_price, entry-exit_price])
                        break
        else:
            # EOD exit (if no SL hit)
            exit_price = day_data.iloc[-1]["Close"]
            if trade_taken:
                if trade_type == "LONG":
                    trades.append([day, trade_type, entry, exit_price, exit_price-entry])
                elif trade_type == "SHORT":
                    trades.append([day, trade_type, entry, exit_price, entry-exit_price])

    # --- Collect trades into DataFrame ---
    trades_df = pd.DataFrame(trades, columns=["Date", "Type", "Entry", "Exit", "PnL_Points"])
    trades_df.set_index("Date", inplace=True)

    # --- Performance Metrics ---
    if trades_df.empty:
        return trades_df, {
            "Total_PnL": 0.0,
            "Max_Drawdown": 0.0,
            "Total_Trades": 0,
            "Win_Rate_pct": 0.0,
            "Avg_PnL": 0.0
        }

    # Reset index to avoid Series misalignment
    cumulative_pnl = trades_df["PnL_Points"].cumsum().reset_index(drop=True)
    running_max = cumulative_pnl.cummax()
    drawdown = cumulative_pnl - running_max
    max_dd = drawdown.min()

    metrics = {
        "Total_PnL": trades_df["PnL_Points"].sum(),
        "Max_Drawdown": float(max_dd),
        "Total_Trades": len(trades_df),
        "Win_Rate_pct": (trades_df["PnL_Points"] > 0).mean() * 100,
        "Avg_PnL": trades_df["PnL_Points"].mean()
    }

    return trades_df, metrics


In [ ]:
trades_df, metrics = backtest_open_move_intraday("^NSEI")
print(metrics)
print(trades_df.head())


/tmp/ipython-input-2382518051.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  daily = yf.download(symbol, start=start, end=end, interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2382518051.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  intraday = yf.download(symbol, start=start, end=end, interval="5m")
[*********************100%***********************]  1 of 1 completed


ValueError: Can only compare identically-labeled Series objects